# **Cell 1 – Imports**

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ensure folders exist (run once)
os.makedirs("../data_raw", exist_ok=True)
os.makedirs("../data_clean", exist_ok=True)
os.makedirs("../Results", exist_ok=True)



# **Cell 2 – Load the data**

In [5]:
cars = pd.read_csv("/content/train.csv")
cars.head()



,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


# **Cell 3 – Handle missing values**

In [9]:

cars_mv = cars.copy()

# Drop rows where target "Price" is missing
cars_mv = cars_mv.dropna(subset=["Price"])

# Identify numeric and categorical columns
num_cols = cars_mv.select_dtypes(include=["int64", "float64"]).columns
cat_cols = cars_mv.select_dtypes(include=["object"]).columns

# Impute numeric → median
for col in num_cols:
    cars_mv[col] = cars_mv[col].fillna(cars_mv[col].median())

# Impute categorical → mode
for col in cat_cols:
    mode = cars_mv[col].mode()
    if not mode.empty:
        cars_mv[col] = cars_mv[col].fillna(mode[0])

# Check missing values
print("Missing values after cleaning:")
print(cars_mv.isna().sum())

# ------------------------------------------------
# SAVE CSV FILE
# ------------------------------------------------
save_path = "../data_clean/data_cleaned_missingValues.csv"
cars_mv.to_csv(save_path, index=False)

print(f"\nCSV saved successfully to: {save_path}")


try:
    from google.colab import files
    files.download(save_path)
    print("\nFile downloaded automatically (Google Colab).")
except:
    from IPython.display import FileLink
    print("\nClick link below to download the file (Jupyter Notebook):")
    FileLink(save_path)



Missing values after cleaning:
Unnamed: 0           0
Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
New_Price            0
Price                0
dtype: int64

CSV saved successfully to: ../data_clean/data_cleaned_missingValues.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


File downloaded automatically (Google Colab).


# **Remove units**

In [11]:
cars_units = cars_mv.copy()

def extract_num(series):
    return series.astype(str).str.extract(r"(\d+\.?\d*)", expand=False).astype(float)

for col in ["Mileage", "Engine", "Power", "New_Price"]:
    if col in cars_units.columns:
        cars_units[col] = extract_num(cars_units[col])

# Save CSV
path_units = "../data_clean/data_units_removed.csv"
cars_units.to_csv(path_units, index=False)

try:
    from google.colab import files
    files.download(path_units)
except:
    from IPython.display import FileLink
    FileLink(path_units)

cars_units.head()



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,4.78,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13.00,1199.0,88.70,5.0,8.61,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,4.78,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,4.78,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08,1461.0,63.10,5.0,4.78,3.50


# **ADD CURRENT AGE FEATURE**

In [12]:
cars_age = cars_units.copy()
CURRENT_YEAR = 2025
cars_age["Current_Age"] = CURRENT_YEAR - cars_age["Year"]

# Save CSV
path_age = "../data_clean/updated_data_with_current_age.csv"
cars_age.to_csv(path_age, index=False)

try:
    from google.colab import files
    files.download(path_age)
except:
    from IPython.display import FileLink
    FileLink(path_age)

cars_age[["Year","Current_Age"]].head()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Year,Current_Age
0,2015,10
1,2011,14
2,2012,13
3,2013,12
4,2013,12


# **ONE-HOT ENCODING FOR FUEL & TRANSMISSION**

In [13]:
cars_cat = cars_age.copy()

encode_cols = []
for col in ["Fuel_Type", "Transmission"]:
    if col in cars_cat.columns:
        encode_cols.append(col)

cars_encoded = pd.get_dummies(cars_cat, columns=encode_cols, drop_first=True)

# Save CSV
path_cat = "../data_clean/Categorical_data_encoded.csv"
cars_encoded.to_csv(path_cat, index=False)

try:
    from google.colab import files
    files.download(path_cat)
except:
    from IPython.display import FileLink
    FileLink(path_cat)

cars_encoded.head()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Current_Age,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,4.78,12.50,10,False,False,True
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,8.61,4.50,14,False,True,True
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,4.78,6.00,13,False,False,True
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,4.78,17.74,12,False,False,False
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,4.78,3.50,12,False,False,True


# **SELECT, FILTER, RENAME, MUTATE, ARRANGE, SUMMARIZE**

In [14]:
# SELECT
selected_df = cars_encoded[["Price", "Year", "Current_Age"]]
display(selected_df.head())

# FILTER
filtered_df = cars_encoded[(cars_encoded["Price"] > 5) & (cars_encoded["Current_Age"] < 8)]
display(filtered_df.head())

# RENAME
renamed_df = cars_encoded.rename(columns={"Price": "Price_Lakh"})
display(renamed_df[["Price_Lakh"]].head())

# MUTATE (create derived variable)
renamed_df["Price_per_Year"] = renamed_df["Price_Lakh"] / (renamed_df["Current_Age"] + 1)
display(renamed_df[["Price_Lakh", "Current_Age", "Price_per_Year"]].head())

# ARRANGE (sorting)
sorted_df = renamed_df.sort_values("Price_Lakh", ascending=False)
display(sorted_df[["Price_Lakh", "Year", "Current_Age"]].head())

# SUMMARIZE (group by transmission)
trans_cols = [c for c in cars_encoded.columns if c.startswith("Transmission_")]

summary_rows = []
for col in trans_cols:
    name = col.replace("Transmission_", "")
    avg_price = cars_encoded.loc[cars_encoded[col] == 1, "Price"].mean()
    summary_rows.append({"Transmission": name, "Avg_Price_Lakh": avg_price})

summary_df = pd.DataFrame(summary_rows)
summary_df


,Price,Year,Current_Age
0,12.50,2015,10
1,4.50,2011,14
2,6.00,2012,13
3,17.74,2013,12
4,3.50,2013,12


,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Current_Age,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual
8,10,Maruti Ciaz Zeta,Kochi,2018,25692,First,21.56,1462.0,103.25,5.0,10.65,9.95,7,False,True,True
26,28,Honda WRV i-VTEC VX,Kochi,2018,37430,First,17.50,1199.0,88.70,5.0,10.57,9.90,7,False,True,True
34,36,Maruti Vitara Brezza ZDi Plus,Kochi,2018,50075,First,24.30,1248.0,88.50,5.0,11.12,8.63,7,False,False,True
65,67,Mercedes-Benz C-Class Progressive C 220d,Coimbatore,2019,15369,First,0.00,1950.0,194.00,5.0,49.14,35.67,6,False,False,False
67,69,Hyundai Creta 1.4 E Plus Diesel,Mumbai,2018,13000,First,22.10,1396.0,88.70,5.0,11.81,10.50,7,False,False,True


,Price_Lakh
0,12.50
1,4.50
2,6.00
3,17.74
4,3.50


,Price_Lakh,Current_Age,Price_per_Year
0,12.50,10,1.136364
1,4.50,14,0.300000
2,6.00,13,0.428571
3,17.74,12,1.364615
4,3.50,12,0.269231


,Price_Lakh,Year,Current_Age
3952,160.00,2017,8
5620,120.00,2011,14
5752,100.00,2015,10
1457,97.07,2019,6
1917,93.67,2018,7


,Transmission,Avg_Price_Lakh
0,Manual,5.415998
